In [20]:
import datetime as dt
from dateutil import parser
import plotly.graph_objects as go
import pandas as pd
import sys
sys.path.append("../")

In [21]:
from api.oanda_api import OandaApi
from infrastructure.instrument_collection import InstrumentCollection as ic
from db.db import DataBaseMongo
pd.set_option('display.max_rows', None)

In [22]:
database = DataBaseMongo()
icol = ic()
api = OandaApi()

In [23]:
load_icol = icol.DB_LoadInstruments()

In [24]:
pairs = []
for pair, val in icol.instruments_dict.items():
    if val.ins_type == "CURRENCY" and "GBP" in pair:
        pairs.append(pair)

In [25]:
pairs[0:5]

['EUR_GBP', 'GBP_NZD', 'GBP_PLN', 'GBP_CHF', 'GBP_CAD']

In [26]:
candle_data = []
for pair in pairs:
    print(pair)
    candle_temp = api.get_candles_df(
        pair_name=pair, granularity='D', count=3000)
    candle_temp['gain'] = (
        (candle_temp.mid_c - candle_temp.mid_o) / candle_temp.mid_o) * 100
    candle_temp['pair'] = pair
    if '_GBP' in pair:
        candle_temp['gain'] = candle_temp['gain'] * -1
    candle_data.append(candle_temp)
candles_df = pd.concat(candle_data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

EUR_GBP
GBP_NZD
GBP_PLN
GBP_CHF
GBP_CAD
GBP_HKD
GBP_ZAR
GBP_JPY
GBP_AUD
GBP_SGD
GBP_USD


In [27]:
candles_df.tail()[0:5]

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,gain,pair
32984,2024-02-06,103924,1.25980,1.26422,1.25945,1.26284,1.25926,1.26413,1.25873,1.26274,1.26033,1.26431,1.25975,1.26293,0.241308,GBP_USD
32985,2024-02-07,103776,1.26260,1.26390,1.25718,1.26184,1.26197,1.26382,1.25709,1.26173,1.26322,1.26398,1.25727,1.26196,-0.060193,GBP_USD
32986,2024-02-08,93910,1.26214,1.26432,1.25990,1.26302,1.26129,1.26424,1.25981,1.26294,1.26298,1.26439,1.25998,1.26311,0.069723,GBP_USD
32987,2024-02-11,79553,1.26269,1.26548,1.26062,1.26282,1.26209,1.26538,1.26053,1.26271,1.26329,1.26558,1.26070,1.26294,0.010295,GBP_USD
32988,2024-02-12,108788,1.26282,1.26922,1.25733,1.25926,1.26225,1.26876,1.25724,1.25915,1.26338,1.26968,1.25741,1.25938,-0.281909,GBP_USD


In [28]:
forex_calendar_data = database.query_all(DataBaseMongo.FOREX_CALENDAR)
forex_calendar_df = pd.DataFrame.from_dict(forex_calendar_data)
forex_calendar_df['date'] = pd.to_datetime(
    forex_calendar_df['date']).dt.date
forex_calendar_df_uk = forex_calendar_df[forex_calendar_df.country == 'united kingdom'].copy(
)

In [29]:
forex_calendar_df_uk[0:5]

,date,country,category,event,symbol,actual,previous,forecast
11,2020-01-13,united kingdom,balance of trade,balance of trade,UKTBTTBA,£4.03B,£-1.34B,£-3.2B
16,2020-01-15,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.3%,1.5%,1.5%
20,2020-01-21,united kingdom,claimant count change,claimant count change,UNITEDKINCLACOUCHA,14.9K,14.9K,26K
34,2020-01-31,united kingdom,consumer confidence,gfk consumer confidence,UKCCI,-9,-11,-10
38,2020-01-31,united kingdom,calendar,brexit deadline,,,,


In [30]:
for column in ['actual', 'previous', 'forecast']:
    for symbol in ['$', 'B', 'K', '£', '¥', '€', '₩', '%']:
        forex_calendar_df_uk[column] = forex_calendar_df_uk[column].str.replace(
            symbol, '')
    forex_calendar_df_uk[column] = forex_calendar_df_uk[column].replace(
        '', '0')
    forex_calendar_df_uk[column] = forex_calendar_df_uk[column].astype(float)

In [31]:
forex_calendar_df_uk[0:5]

,date,country,category,event,symbol,actual,previous,forecast
11,2020-01-13,united kingdom,balance of trade,balance of trade,UKTBTTBA,4.03,-1.34,-3.2
16,2020-01-15,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.30,1.50,1.5
20,2020-01-21,united kingdom,claimant count change,claimant count change,UNITEDKINCLACOUCHA,14.90,14.90,26.0
34,2020-01-31,united kingdom,consumer confidence,gfk consumer confidence,UKCCI,-9.00,-11.00,-10.0
38,2020-01-31,united kingdom,calendar,brexit deadline,,0.00,0.00,0.0


In [32]:
forex_calendar_df_uk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 274 entries, 11 to 2225
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      274 non-null    object 
 1   country   274 non-null    object 
 2   category  274 non-null    object 
 3   event     274 non-null    object 
 4   symbol    274 non-null    object 
 5   actual    274 non-null    float64
 6   previous  274 non-null    float64
 7   forecast  274 non-null    float64
dtypes: float64(3), object(5)
memory usage: 19.3+ KB


In [33]:
date_match = {}
for original_date in forex_calendar_df_uk.date.unique():
    matched_date = original_date
    tries = 0
    while matched_date not in candles_df.time.values:
        matched_date = matched_date + dt.timedelta(days=1)
        tries += 1
        if tries > 5:
            print("Failed to find a matching date.")
            break
    date_match[original_date] = matched_date

In [34]:
forex_calendar_df_uk['original_date'] = forex_calendar_df_uk['date']
forex_calendar_df_uk['date'] = [date_match[original_date]
                                for original_date in forex_calendar_df_uk['date']]

In [37]:
forex_calendar_df_uk.head(10)

,date,country,category,event,symbol,actual,previous,forecast,original_date,delta_previous,delta_forecast
11,2020-01-13,united kingdom,balance of trade,balance of trade,UKTBTTBA,4.030,-1.340,-3.2,2020-01-13,5.370,7.230
16,2020-01-15,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.300,1.500,1.5,2020-01-15,-0.200,-0.200
20,2020-01-21,united kingdom,claimant count change,claimant count change,UNITEDKINCLACOUCHA,14.900,14.900,26.0,2020-01-21,0.000,-11.100
34,2020-02-02,united kingdom,consumer confidence,gfk consumer confidence,UKCCI,-9.000,-11.000,-10.0,2020-01-31,2.000,1.000
38,2020-02-02,united kingdom,calendar,brexit deadline,,0.000,0.000,0.0,2020-01-31,0.000,0.000
49,2020-02-11,united kingdom,balance of trade,balance of trade,UKTBTTBA,7.715,1.821,-2.6,2020-02-11,5.894,10.315
55,2020-02-18,united kingdom,claimant count change,claimant count change,UNITEDKINCLACOUCHA,5.500,2.600,15.0,2020-02-18,2.900,-9.500
58,2020-02-19,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.800,1.300,1.4,2020-02-19,0.500,0.400
68,2020-03-01,united kingdom,consumer confidence,gfk consumer confidence,UKCCI,-7.000,-9.000,-8.0,2020-02-28,2.000,1.000
88,2020-03-11,united kingdom,interest rate,boe press conference,,0.000,0.000,0.0,2020-03-11,0.000,0.000


In [36]:
forex_calendar_df_uk['delta_previous'] = forex_calendar_df_uk['actual'] - \
    forex_calendar_df_uk['previous']
forex_calendar_df_uk['delta_forecast'] = forex_calendar_df_uk['actual'] - \
    forex_calendar_df_uk['forecast']

In [38]:
forex_calendar_df_uk.head(4)

,date,country,category,event,symbol,actual,previous,forecast,original_date,delta_previous,delta_forecast
11,2020-01-13,united kingdom,balance of trade,balance of trade,UKTBTTBA,4.03,-1.34,-3.2,2020-01-13,5.37,7.23
16,2020-01-15,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.30,1.50,1.5,2020-01-15,-0.20,-0.20
20,2020-01-21,united kingdom,claimant count change,claimant count change,UNITEDKINCLACOUCHA,14.90,14.90,26.0,2020-01-21,0.00,-11.10
34,2020-02-02,united kingdom,consumer confidence,gfk consumer confidence,UKCCI,-9.00,-11.00,-10.0,2020-01-31,2.00,1.00


In [39]:
candles_df.head(2)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,gain,pair
0,2012-09-13,13937,0.80405,0.81149,0.80374,0.80931,0.80395,0.81143,0.80368,0.80916,0.80415,0.81155,0.80380,0.80946,-0.654188,EUR_GBP
1,2012-09-15,176,0.80931,0.80938,0.80844,0.80888,0.80881,0.80888,0.80794,0.80838,0.80981,0.80988,0.80894,0.80938,0.053132,EUR_GBP


In [40]:
candles_analyze = candles_df[['time', 'pair', 'gain']].copy()

In [42]:
candles_analyze.head(2)

,time,pair,gain
0,2012-09-13,EUR_GBP,-0.654188
1,2012-09-15,EUR_GBP,0.053132


In [43]:
merged_dataframe = pd.merge(

    left=candles_analyze, right=forex_calendar_df_uk, left_on='time', right_on='date')

In [45]:
merged_dataframe.head(2)

,time,pair,gain,date,country,category,event,symbol,actual,previous,forecast,original_date,delta_previous,delta_forecast
0,2020-01-13,EUR_GBP,0.319586,2020-01-13,united kingdom,balance of trade,balance of trade,UKTBTTBA,4.03,-1.34,-3.2,2020-01-13,5.37,7.23
1,2020-01-15,EUR_GBP,0.413987,2020-01-15,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.30,1.50,1.5,2020-01-15,-0.20,-0.20


In [46]:
merged_dataframe.category.unique()

array(['balance of trade', 'inflation rate', 'claimant count change',
       'consumer confidence', 'calendar', 'interest rate',
       'retail sales mom', 'composite pmi', 'monthly gdp mom',
       'monthly gdp yoy', 'manufacturing pmi', 'services pmi',
       'unemployment rate', 'gdp growth rate', 'gdp annual growth rate'],
      dtype=object)

In [50]:
merged_catagory_dataframe = merged_dataframe[merged_dataframe.category == 'inflation rate']

In [ ]:
merged_catagory_dataframe[0:5]